In [159]:
import pandas as pd
import numpy as np

In [160]:
text=pd.read_csv("coupons_ner.csv",header=None)

In [161]:
text

,0
0,Save $2.00 ONE Downy Liquid Fabric Conditioner...
1,Save $2.00 ONE Tide PODS OR Tide Power PODS (e...
2,Save $2.00 ONE Tide Laundry Detergent (exclude...
3,SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...
4,$3.00 OFF when you purchase any THREE (3) Pepp...
...,...
881,SAVE $2.00 when you buy $7 worth of participat...
882,SAVE $5.00 on any Gain scented purchase of $30...
883,SAVE $1.00 when you buy any ONE (1) NEW AXE pr...
884,BUY TWO GET ONE Buy ONE (1) CHIPS AHOY! Cookie...


### Exercise 1 - Returning the FaceValue as Output

In [162]:
import re

In [163]:
text[0][0]

'Save $2.00 ONE Downy Liquid Fabric Conditioner 72 ld or larger (includes Downy Odor Protect 48 oz or larger OR Downy Wrinkle Guard 40 oz or larger OR Downy Nature Blends 67 oz or larger) OR Bounce/Downy Sheets 130 ct or larger (includes Bounce/Downy Wrinkle Guard 80 ct or larger) OR In Wash Scent Boosters 8.6 oz or larger (includes Downy Unstopables, Fresh Protect, Odor Protect, and Infusions) (excludes Downy Libre Enjuague, Gain Fireworks, and trial/travel size).'

Using Regex to separate the currency/facevalue

Logic - 
Starts with a '$', 1 or more digits, a decimal point(\.), 0 or more digits.

or
Digits followed by cents

or
Single digit after $

or
Decimal after $ followed by 1 or more digits

In [170]:
def find_FV(text):
    if type(text)==str:
        return(re.findall('\$[0-9]+\.[0-9]*|[0-9]+¢|\$[0-9]+|\$\.[0-9]+',text))
    else:
        return([re.findall('\$[0-9]+\.[0-9]*|[0-9]+¢|\$[0-9]+|\$\.[0-9]+',text[0][i]) for i in range(0,len(text))])

In [166]:
FaceValue=find_FV('Save $2.00 ONE Downy Liquid Fabric Conditioner 72 ld or larger (includes Downy Odor Protect 48 oz or larger OR Downy Wrinkle Guard 40 oz or larger OR Downy Nature Blends 67 oz or larger) OR Bounce/Downy Sheets 130 ct or larger (includes Bounce/Downy Wrinkle Guard 80 ct or larger) OR In Wash Scent Boosters 8.6 oz or larger (includes Downy Unstopables, Fresh Protect, Odor Protect, and Infusions) (excludes Downy Libre Enjuague, Gain Fireworks, and trial/travel size).')
FaceValue

['$2.00']

In [181]:
find_FV("where are my $0.50")

['$0.50']

In [182]:
find_FV("please lend me $.56")

['$.56']

In [176]:
#Extracting the FaceValue from all the entries
FV_all=find_FV(text)

In [178]:
#To check how many entries did not return a FaceValue and their corresponding index
s=0
c=[]
for i in range(0,len(FV_all)):
    if FV_all[i]==[]:
        c.append(i)
        s=s+1

In [179]:
s

273

There are 273 null empty lists in FaceValue.
These are entries which do not contain any FaceValue text

# TASK 2 - Find the product names using NER

# NER

In [183]:
#!pip install spacy

In [184]:
#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz


In [254]:
import spacy
from tqdm import tqdm
nlp=spacy.load('en')

In [255]:
d1=nlp("What is his Name Apple")
for t in d1.ents:
    print(t.text, t.start_char, t.end_char, t.label_)

Apple 17 22 ORG


In [256]:
#CHecking to see how the base spacy 'en' model does on the given text
d2= nlp('Downy Liquid Fabric Conditioner 72 ld or larger (includes Downy Odor Protect 48 oz or larger OR Downy Wrinkle Guard 40 oz or larger OR Downy Nature Blends 67 oz or larger) OR Bounce/Downy Sheets 130 ct or larger (includes Bounce/Downy Wrinkle Guard 80 ct or larger) OR In Wash Scent Boosters 8.6 oz or larger (includes Downy Unstopables, ')
for t in d2.ents:
    print(t.text, t.start_char, t.end_char, t.label_)

Downy Liquid Fabric Conditioner 0 31 ORG
72 32 34 DATE
Downy Odor Protect 58 76 LOC
48 77 79 DATE
Downy Wrinkle Guard 40 96 118 WORK_OF_ART
Downy Nature Blends 135 154 WORK_OF_ART
67 155 157 DATE
Bounce/Downy Wrinkle Guard 80 222 251 EVENT
Downy Unstopables 319 336 ORG


The base model does not perform well.

We can build a training corpus for custom NER using spacy.

In [302]:
# Building a corpus for training the model - 
train=[
    ('ONE Downy Liquid Fabric Conditioner', {'entities':[(4,34, 'PRODUCT')]}),
    ('72 ld or larger (includes Downy Odor Protect 48 oz or larger OR Downy Wrinkle Guard 40 oz', {'entities':[(26,43, 'PRODUCT'),(64,82, 'PRODUCT')]}),
    ('Downy Unstopables', {'entities':[(0,16, 'PRODUCT')]}),
    ('Downy Nature Blends', {'entities':[(0,18, 'PRODUCT')]}),
    ('Bounce/Downy Sheets', {'entities':[(0,18, 'PRODUCT')]}),
    ('Bounce/Downy Wrinkle Guard includes ', {'entities':[(0,25, 'PRODUCT')]}),
    ('Included/excludes/whatever Downy Libre Enjuague', {'entities':[(27,46, 'PRODUCT')]}),
]

In [303]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path

In [304]:
@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    output_dir=("C:\Users\asus\Desktop\0.Coding Assignment\NER-Task", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int))

SyntaxError: unexpected EOF while parsing (<ipython-input-304-d9c61233f5fe>, line 4)

In [305]:
model=None
output_dir=Path("C:\\Users\\asus\\Desktop\\0.Coding Assignment\\NER-Task")
n_iter=100

In [306]:
#Loading the model (if the model is not present it creates a 'blank' model)
if model is not None:
    nlp = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")

Created blank 'en' model


In [307]:
# create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe('ner')

Training the NER

In [ ]:
#Adding the label to our ner
for i in range(0,len(train)):
    ner.add_label('PRODUCT')

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(train)
        losses = {}
        for text, annotations in tqdm(train):
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)



  0%|                                                                                            | 0/7 [00:00<?, ?it/s]

 14%|████████████                                                                        | 1/7 [00:00<00:01,  4.01it/s]

 29%|████████████████████████                                                            | 2/7 [00:00<00:01,  4.50it/s]

 43%|████████████████████████████████████                                                | 3/7 [00:00<00:00,  5.02it/s]

 57%|████████████████████████████████████████████████                                    | 4/7 [00:00<00:00,  5.62it/s]

 86%|████████████████████████████████████████████████████████████████████████            | 6/7 [00:00<00:00,  6.58it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00,  7.07it/s]


  0%|                                                                                            | 0/7 [00:00<?, ?it/s]

 14%|████████████            

{'ner': 19.057351276278496}




 29%|████████████████████████                                                            | 2/7 [00:00<00:00,  8.14it/s]

 57%|████████████████████████████████████████████████                                    | 4/7 [00:00<00:00,  8.53it/s]

 71%|████████████████████████████████████████████████████████████                        | 5/7 [00:00<00:00,  8.83it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00,  9.24it/s]


  0%|                                                                                            | 0/7 [00:00<?, ?it/s]

{'ner': 14.174307689070702}




 29%|████████████████████████                                                            | 2/7 [00:00<00:00,  9.29it/s]

 57%|████████████████████████████████████████████████                                    | 4/7 [00:00<00:00,  8.87it/s]

 86%|████████████████████████████████████████████████████████████████████████            | 6/7 [00:00<00:00,  9.08it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00,  8.91it/s]


  0%|                                                                                            | 0/7 [00:00<?, ?it/s]

 29%|████████████████████████                                                            | 2/7 [00:00<00:00, 11.15it/s]

...



 43%|████████████████████████████████████                                                | 3/7 [00:00<00:00,  7.87it/s]

 57%|████████████████████████████████████████████████                                    | 4/7 [00:00<00:00,  8.05it/s]

 86%|████████████████████████████████████████████████████████████████████████            | 6/7 [00:00<00:00,  8.60it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00,  9.35it/s]


  0%|                                                                                            | 0/7 [00:00<?, ?it/s]

 14%|████████████                                                                        | 1/7 [00:00<00:00,  9.50it/s]

{'ner': 2.6335632008231917e-13}




 43%|████████████████████████████████████                                                | 3/7 [00:00<00:00,  9.74it/s]

 57%|████████████████████████████████████████████████                                    | 4/7 [00:00<00:00,  9.79it/s]

 71%|████████████████████████████████████████████████████████████                        | 5/7 [00:00<00:00,  8.86it/s]

 86%|████████████████████████████████████████████████████████████████████████            | 6/7 [00:00<00:00,  8.70it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00,  8.81it/s]


  0%|                                                                                            | 0/7 [00:00<?, ?it/s]

{'ner': 6.418275544492631e-10}




 29%|████████████████████████                                                            | 2/7 [00:00<00:00,  9.25it/s]

 57%|████████████████████████████████████████████████                                    | 4/7 [00:00<00:00,  9.21it/s]

 71%|████████████████████████████████████████████████████████████                        | 5/7 [00:00<00:00,  8.86it/s]